In [ ]:
!pip install  torch tqdm tqdm

In [1]:
import os
import json
from tqdm import tqdm

In [2]:
data_path = 'e:\\'

In [3]:
train_set = []
with open(os.path.join(data_path, 'train_set_full.jl'), 'r') as f:
    for l in tqdm(f):
        d = json.loads(l)
        train_set.append((d[0][1:], d[1][1:], d[2]))

185472it [04:48, 642.92it/s]


In [4]:
dev_set = []
with open(os.path.join(data_path, 'dev_set_full.jl'), 'r') as f:
#     for i in range(5000):
    for l in f:
#         l = f.readline()
        d = json.loads(l)
        dev_set.append((d[0][1:], d[1][1:], d[2]))

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm
import time
import random
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
import sklearn.metrics
device = torch.device('cpu') # torch.device('cuda') 
max_len1 = 30
max_len2 = 10
batch_size = 512
data_workers = 4

In [6]:
class MyDataSet(torch.utils.data.Dataset):
    def __init__(self, examples):
        self.examples = examples

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, index):
        example = self.examples[index]
        s1 = example[0]
        s2 = example[1]
        label = example[2]
        l1 = len(s1)
        l2 = len(s2)
        if l1 < max_len1:
            s1 += [[0] * 78 for _ in range(max_len1 - l1)]
        if l2 < max_len2:
            s2 += [[0] * 78 for _ in range(max_len2 - l2)]
        return s1, l1, s2, l2, label, index

def the_collate_fn(batch):
    #print(batch)
    s1 = [b[0] for b in batch]
    s2 = [b[2] for b in batch]
    l1 = [b[1] for b in batch]
    l2 = [b[3] for b in batch]
    label = torch.Tensor([b[4] for b in batch])
    indexs = [b[5] for b in batch]
#     print(type(s1), len(s1))
#     for p in s1:
#         print(type(p), len(p))
    s1 = torch.FloatTensor(s1)
    s2 = torch.FloatTensor(s2)
    return s1, l1,s2, l2, label, indexs

In [7]:
train_dataset = MyDataSet(train_set)
train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle = True,
    num_workers=data_workers,
    collate_fn=the_collate_fn,
)



dev_dataset = MyDataSet(dev_set)
dev_data_loader = torch.utils.data.DataLoader(
    dev_dataset,
    batch_size=batch_size,
    shuffle = True,
    num_workers=data_workers,
    collate_fn=the_collate_fn,
)

In [8]:
class DoubleLSTMClassify(nn.Module):
    def __init__(self, embedding_dim=78, hidden_dim=256):
        super(DoubleLSTMClassify, self).__init__()
        self.hidden_dim = hidden_dim
#         self.embedding = nn.Embedding(word_size, embedding_dim)
#         self.embedding.from_pretrained(torch.FloatTensor(vec))
        self.lstm1 = nn.LSTM(embedding_dim, hidden_dim // 2, num_layers=3, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(embedding_dim, hidden_dim // 2, num_layers=3, bidirectional=True, batch_first=True)
        self.cls = nn.Linear(hidden_dim*2, 1)
        self.sigmoid = nn.Sigmoid()
        
        
    def forward(self, s1, length1, s2, length2, label=None):
        batch_size = s1.shape[0]
#         b1 = self.embedding(s1)
#         b2 = self.embedding(s2)
        l1 = self.lstm1(s1)[0]
        l2 = self.lstm2(s2)[0]
        x = torch.zeros(batch_size, self.hidden_dim*2).to(device)
        for b in range(batch_size):
            x[b][:self.hidden_dim] = l1[b][length1[b]-1]
            x[b][self.hidden_dim:] = l2[b][length2[b]-1]
        r = self.cls(x)
        r = self.sigmoid(r)
        if label != None:
            criterion = nn.BCELoss()
            return criterion(r.squeeze(1), label)
        return r

In [9]:
def evalx(pred, truth, th):
    predx = []
    for x in pred:
        if x > th:
            predx.append(1)
        else:
            predx.append(0)
    prec,rec,f1,num = precision_recall_fscore_support(truth,predx, average="binary")
    accuracy = sklearn.metrics.accuracy_score(truth, predx)
    return prec, rec, f1, accuracy


def eval(th=0.5):
    predv = []
    truth =[]
    for batch in tqdm(dev_data_loader):
        with torch.no_grad():
    #         model.test()
            s1, l1, s2, l2, label, index = batch
            s1 = s1.to(device)
            s2 = s2.to(device)
            label = label.to(device)
            r = model(s1, l1, s2, l2)
            truth += label
#             pred += predx
            predv += r
    for i in range(len(truth)):
        truth[i] = int(truth[i].cpu().item())
    if th is not None:
        maxp, maxr,maxf1, maxa = evalx(predv, truth, th)
        maxth = th
    else:
        maxf1 = 0
        maxp, maxr, maxa, maxth = 0,0,0,0
        for t in tqdm(predv + [max(predv)+1]):
            prec, rec, f1, accuracy = evalx(predv, truth, t)
            if f1 > maxf1:
                print(prec, rec, f1, accuracy)
                maxp, maxr,maxf1, maxa, maxth = prec, rec, f1, accuracy, t
#     prec,rec,f1,num = precision_recall_fscore_support(truth,pred, average="binary")
#     accuracy = sklearn.metrics.accuracy_score(truth, pred)
#     print(len(truth), sum(pred), sum(truth))
#     print('p=%.4f\tr=%.4f\tf1=%.4f\ta=%.4f' % (prec,rec,f1, accuracy))
    return maxp, maxr, maxf1, maxth, predv, truth

In [10]:
model = DoubleLSTMClassify(78)
model.to(device)
learning_rate = 0.2#0.01
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,  milestones = [20, 40, 50,60, 70, 80, 90], gamma = 0.5, last_epoch=-1)

In [11]:
def get_res(pred, trut):
    predv = [x.cpu().item() for x in pred]
    trutv = trut
    if True:
        pos_l = []
        neg_l = []
        for i in range(len(trutv)):
            if trutv[i] == 1:
                pos_l.append(predv[i])
            else:
                neg_l.append(predv[i])
        pos_l.sort()
        neg_l.sort()
        tar = list(set(predv))
        tar.sort()
        tar = [0] + tar + [tar[-1] + 1]
        i, j = 0, 0
        rl = []
        
        for x in tar:
            while  i < len(pos_l) and pos_l[i] < x:
                i += 1
            while j < len(neg_l) and neg_l[j] < x:
                j += 1
            tp = len(pos_l) - i
            fp = len(neg_l) - j
            tn = j
            fn = i
            precision = 0 if (tp+fp) == 0 else tp / (tp+fp)
            recall = 0 if (tp + fn) == 0 else tp / (tp + fn)
            f1 = 0 if (precision+recall) == 0 else  2*(precision*recall) / (precision+recall)
            rl.append([precision, recall, f1])
        rl.sort(key = lambda x: (-x[2], -x[0], -x[1]))
        r1 = rl[::]
        rl.sort(key = lambda x: (-x[0], -x[1], -x[2]))
        r2 = rl[::]
        return r1, r2

In [12]:
!mkdir full_train-1

A subdirectory or file full_train-1 already exists.


In [13]:
loss_list = []
bf15 = 0
bf1a = 0
import time
def log(x):
    with open('full-1.log', 'a') as f:
        f.write(str(time.time()) + '\n')
        f.write(x + '\n')
    print(x)

In [ ]:
i = 0
loss_sum = 0
for batch in tqdm(train_data_loader):
    break

  0%|                                                                                          | 0/363 [00:00<?, ?it/s]

In [ ]:
loss_list = []
bf15 = 0
bf1a = 0
import time
def log(x):
    with open('full-1.log', 'a') as f:
        f.write(str(time.time()) + '\n')
        f.write(x + '\n')
    print(x)
# result_list = []
for b in range(40):
    print('epoch', b)
    i = 0
    loss_sum = 0
    for batch in tqdm(train_data_loader):
        i += 1
        model.train()
        optimizer.zero_grad()
        s1, l1, s2, l2, label, index = batch
        s1 = s1.to(device)
        s2 = s2.to(device)
        label = label.to(device)
        loss = model(s1, l1,s2, l2, label)
        loss_sum += loss.item()
        loss.backward()
        optimizer.step()
#         scheduler.step()
    xresult = eval()
    r1, r2 = get_res(*xresult[4:])
    save_flag = False
    if xresult[2] > bf15:
        bf15 = xresult[2]
        save_flag = True
    elif r1[0][2] > bf1a:
        bf1a = r1[0][2]
        save_flag = True
    if save_flag:
        torch.save(model, 'full_train-1/model_20201223_%.2f_%.2f_epoch_%d_layer_3.pkl' % (xresult[2], r1[0][2], b))
    log(str(('train loss', loss_sum / i, xresult[:4], bf15, bf1a)))
    loss_list.append(loss_sum / i)
#     result_list.append(xresult)
    print()
#     eval(0.0006)

  0%|                                                                                          | 0/363 [00:00<?, ?it/s]

epoch 0
